# Evo Client Common v2

`evo-client-common`

Evo Client Common is a Python package that establishes a common framework for use by client libraries that interact
with Evo APIs. The v2 version of the package is a complete rewrite of the original package, and is designed to improve
the developer experience by providing a more intuitive and flexible API. This notebook demonstrates how to use the
v2 package to interact with Evo APIs.

## ITransport

The `ITransport` interface is used to make HTTP requests to Evo APIs. The `AioTransport` class is an implementation
based on the `aiohttp` library, which is an optional dependency. Different HTTP client libraries can be substituted by
implementing a facade that implements the `ITransport` interface.

Transport objects must be re-entrant so that they can be used by multiple coroutines at the same time. `AioTransport`
uses an internal counter to track the number of places where the transport is being used. When the counter reaches zero,
the underlying HTTP client session is closed, and any related resources are released. The next time the transport is
opened, a new session will be created.

In [ ]:
from evo.aio import AioTransport
from evo.common.utils import BackoffIncremental

# Configure the transport.
transport = AioTransport(
    user_agent="evo-client-common-poc",
    max_attempts=3,
    backoff_method=BackoffIncremental(2),
    num_pools=4,
    verify_ssl=True,
)

# We can open the transport outside a context manager so that the underlying session is left open. This can save
# time if we are going to make multiple batches of requests in the same area of code. Ideally, the transport should
# be closed when it is no longer needed.
await transport.open()

## Logging in to Evo

The `IAuthorizer` interface is used to authenticate with Evo APIs, by automatically attaching the default headers to
API requests. The `AuthorizationCodeAuthorizer` class is an OAuth implementation of `IAuthorizer`, utilizing a reference OAuth
implementation that is built using the `aiohttp` library. `aiohttp` is an optional dependency, so it must be installed
for the `AuthorizationCodeAuthorizer` implementation to work.

In [ ]:
from evo.oauth import OIDCConnector, AuthorizationCodeAuthorizer

ISSUER_URI = "https://qa-ims.bentley.com"
REDIRECT_URL = "http://localhost:3000/signin-oidc"
CLIENT_NAME = "EvoPythonSDK"
CLIENT_ID = CLIENT_NAME.lower()

authorizer = AuthorizationCodeAuthorizer(
    redirect_url=REDIRECT_URL,
    oidc_connector=OIDCConnector(
        transport=transport,
        oidc_issuer=ISSUER_URI,
        client_id=CLIENT_ID,
    ),
)

# Login to the Evo platform.
await authorizer.login()

Alternatively, a client of `client credientials` grant type can use the `ClientCredentialsAuthorizer` for authorization into Evo. This allows for service to service requests, instead of user login and redirects. 

In [ ]:
from evo.oauth import OIDCConnector, ClientCredentialsAuthorizer, OAuthScopes

ISSUER_URI = "https://qa-ims.bentley.com"
CLIENT_NAME = "<client_name>"
CLIENT_SECRET = "<client_secret"
CLIENT_ID = CLIENT_NAME.lower()

authorizer = ClientCredentialsAuthorizer(
    oidc_connector=OIDCConnector(
        transport=transport,
        oidc_issuer=ISSUER_URI,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
    ),
    scopes=OAuthScopes.all_evo,
)

# Authorize the client.
await authorizer.authorize()

## Listing Organizations

In most user-facing environments it will be necessary to list the organizations that the user has access to. The
`DiscoveryApiCLient` interacts with the Discovery API to retrieve this information. Simply give it a connector
pointing to the appropriate host, and it will do the rest.

In [ ]:
from evo.common import ApiConnector
from evo.discovery import DiscoveryApiClient

# Select an Organization.
async with ApiConnector("https://uat-api.test.seequent.systems", transport, authorizer) as idp_connector:
    discovery_client = DiscoveryApiClient(idp_connector)
    organizations = await discovery_client.list_organizations()

# Select the first organization for this example.
selected_organization = organizations[0]
print("Selected organization:", selected_organization.display_name)

## Listing Workspaces

Once an organization has been selected, the next step is to list the workspaces that the user has access to. Each
organization _may_ span multiple hubs, although there is a current commitment to a 1:1 relationship in production
environments.

We will create a connector targeting the hub URL, which we can reuse later for talking to individual services. The
transport and authorizer objects are also reused.

In [ ]:
from evo.workspaces import WorkspaceServiceClient

hub = next(hub for hub in selected_organization.hubs if hub.code == "350mt")  # Using the 350mt hub for this example.

# This connector can be used to connect to any service supported by the hub.
hub_connector = ApiConnector(hub.url, transport, authorizer)

# List the workspaces.
async with hub_connector:
    workspace_client = WorkspaceServiceClient(hub_connector, selected_organization.id)
    workspaces = await workspace_client.list_workspaces()

# Select the first workspace for this example.
selected_workspace = workspaces[0]
print("Selected workspace:", selected_workspace.display_name)

## Interacting with Service Clients

The `Workspace` object can generate an `Environment`, which contains the organization and workspace IDs, and can be
used to resolve cache locations. Evo Client Common does not implement any specific service clients*, but it provides
a `BaseServiceClient` type that should be used as a base class for service clients.

The `BaseServiceClient` defines a shared constructor for service clients, as well as convenient cache management via the `cache` property, and the `clear_cache()` method. 

In [ ]:
from evo.common import BaseServiceClient

## Interact with a service.
async with hub_connector:
    service_client = BaseServiceClient(selected_workspace.get_environment(), hub_connector)
    ...  # Do something with the service client.